## LCEL 인터페이스


사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


LCEL 문법을 사용하여 chain 을 생성합니다.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

In [5]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대하여 3문장으로 설명해줘.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x14a0b83d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x14a0c3b90>, root_client=<openai.OpenAI object at 0x13abfd690>, root_async_client=<openai.AsyncOpenAI object at 0x14a0cd2d0>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

## stream: 실시간 출력


- 이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고,
- 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. 
- `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 

In [6]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

멀티모달은 여러 가지 다양한 형태의 미디어를 결합하여 사용자에게 정보를 전달하는 방식이다. 이는 텍스트, 이미지, 음성, 동영상 등을 함께 활용하여 더 풍부한 경험을 제공할 수 있게 한다. 멀티모달은 사용자들에게 보다 효과적인 커뮤니케이션과 이해를 도와주며, 정보를 보다 효과적으로 전달할 수 있는 장점을 가지고 있다.

## invoke: 호출


`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [7]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 자연어 처리 기술을 기반으로 한 대화형 인공지능 모델로, 다양한 주제에 대한 대화를 이어나가거나 질문에 답변하는 데 사용됩니다. 사용자들은 ChatGPT를 통해 실시간 대화를 나누거나 정보를 얻을 수 있으며, 문제 해결이나 창의적인 아이디어 발전에도 활용될 수 있습니다. ChatGPT는 계속 발전하고 있는 기술로, 사용자들과의 상호작용을 통해 더욱 똑똑해지고 유용한 서비스를 제공하고 있습니다.'

## batch: 배치(단위 실행)

- 한 번에 여러 개의 요청을 처리할 때 유리

함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다.

In [8]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 OpenAI가 개발한 자연어 처리 모델로, 대화형 인공 지능 채팅을 가능하게 합니다. 사용자의 입력을 분석하여 자연스러운 대화를 이어가는 데 중점을 두고 있으며, 다양한 주제에 대한 정보를 제공하거나 상황에 맞는 답변을 제공합니다. ChatGPT는 사람과 자연스럽게 소통하는 데 도움이 되는 혁신적인 기술입니다.',
 'Instagram은 사진 및 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 쉽게 공유하고 소통할 수 있습니다. 해시태그를 통해 관심사에 맞는 콘텐츠를 찾을 수 있고, 팔로우를 통해 친구들과 즐거운 소통을 즐길 수 있습니다. 또한 인스타그램 스토리를 통해 임시적인 콘텐츠를 공유하고 일상을 더욱 생생하게 전달할 수 있습니다.']

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

`config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [9]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 인공지능 챗봇으로, 자연어 처리 기술을 활용하여 대화를 수행합니다. 사용자의 질문에 응답하거나 대화를 이어가는 등 다양한 상황에서 활용됩니다. ChatGPT를 통해 사용자는 쉽게 정보를 얻거나 문제를 해결할 수 있습니다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 시각적으로 공유하고 소통할 수 있습니다. 해시태그를 통해 관심사나 주제별로 게시물을 찾을 수 있으며, 좋아요나 댓글을 통해 다른 사용자와 소통할 수 있습니다. 인스타그램 스토리 기능을 통해 24시간 동안 사라지는 재미있는 콘텐츠를 공유할 수도 있습니다.',
 '멀티모달은 여러 가지 방식으로 정보를 전달하는 것을 의미합니다. 이는 텍스트, 이미지, 음성, 동영상 등 다양한 매체를 활용하여 사용자에게 다양한 경험을 제공하는 것을 의미합니다. 멀티모달을 통해 사용자는 보다 풍부한 정보를 받을 수 있고, 상호작용이 향상되어 더 나은 사용자 경험을 제공할 수 있습니다.',
 '프로그래밍은 컴퓨터에게 작업을 시키기 위해 명령을 작성하는 과정이다. 이러한 명령은 프로그래밍 언어로 작성되며, 컴퓨터가 이를 이해하여 실행한다. 프로그래밍을 통해 다양한 소프트웨어 및 애플리케이션을 개발할 수 있으며, 이는 현대 사회에서 매우 중요한 역할을 한다.',
 '머신러닝은 컴퓨터가 데이터를 분석하고 패턴을 학습하여 스스로 문제를 해결하는 인공지능 기술이다. 이를 통해 예측, 분류, 군집화 등 다양한 작업을 수행할 수 있으며, 빅데이터 시대에 매우 중요한 역할을 한다. 머신러닝은 알고리즘을 통해 데이터를 분석하고 모델을 학습시켜 예측력을 향상시키는 과정으로 이루어진다.']

## async stream: 비동기 스트림


함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print` 함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.


In [10]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 동영상을 공유하고 시청할 수 있는 온라인 동영상 플랫폼이다. 사용자들은 다양한 주제의 동영상을 업로드하고 시청할 수 있으며, 댓글을 통해 소통할 수 있다. 또한 광고 수익을 얻을 수 있는 유튜브 파트너십 프로그램도 있다.

## async invoke: 비동기 호출


`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.


In [11]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

In [12]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

'NVDA는 미국의 컴퓨터 기술 기업인 NVIDIA Corporation의 주식 코드이다. 이 회사는 그래픽 처리 장치(GPU)를 제조하며, 주로 게임 산업 및 인공지능 분야에서 활약하고 있다. NVDA 주식은 기술 기업 투자자들 사이에서 인기 있는 투자 대상 중 하나이다.'

## async batch: 비동기 배치


함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 `chain` 객체의 `abatch` 메서드를 사용하여 `topic` 에 대한 작업을 비동기적으로 처리하고 있습니다.

`await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.


In [13]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [14]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process

['YouTube는 동영상을 업로드, 시청 및 공유할 수 있는 온라인 동영상 플랫폼이다. 사용자는 다양한 주제의 동영상을 시청하고 댓글을 달아 소통할 수 있다. 또한 유명한 크리에이터들이 영상을 제작하고 수익을 창출하는데 이용되기도 한다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상을 시각적으로 공유하고 다른 사람들과 소통할 수 있다. 해시태그를 통해 관심사나 주제별로 컨텐츠를 검색하고 팔로우할 수 있어 다양한 콘텐츠를 쉽게 찾을 수 있다. 인스타그램 스토리와 라이브 기능을 통해 실시간으로 소통하고 다양한 콘텐츠를 제공하며, 사용자들은 자신의 삶을 더욱 다채롭게 공유할 수 있다.',
 '페이스북은 세계적으로 가장 대중적인 소셜 네트워크 서비스이다. \n사용자들은 친구들과 가족들과 소통하고, 사진, 동영상, 이야기 등을 공유할 수 있다. \n또한 광고 플랫폼으로도 이용되어 다양한 기업들이 제품이나 서비스를 홍보하는데 사용된다.']

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때, 각 요소를 병렬로 실행합니다.


`langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

`ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `country`에 대한 **수도** 와 **면적** 을 구하는 두 개의 체인(`chain1`, `chain2`)을 만듭니다.

이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결됩니다. 마지막으로, `RunnableParallel` 클래스를 사용하여 이 두 체인을 `capital`와 `area`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성합니다.


In [16]:
# Runnable : 프롬프트, 모델, 아웃풋 파서가 될 수 있다. (+ 리트리버, 도구)
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (  # 러너블을 묶어서 체인을 구성한다.
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
# 파라미터의 의미는 -> 각 답변에 접근하기 위한 Key
combined = RunnableParallel(capital=chain1, area=chain2)

`chain1.invoke()` 함수는 `chain1` 객체의 `invoke` 메서드를 호출합니다.

이때, `country`이라는 키에 `대한민국`라는 값을 가진 딕셔너리를 인자로 전달합니다.


In [17]:
# chain1 를 실행합니다.
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울이에요.'

이번에는 `chain2.invoke()` 를 호출합니다. `country` 키에 다른 국가인 `미국` 을 전달합니다.


In [18]:
# chain2 를 실행합니다.
chain2.invoke({"country": "미국"})

'미국의 면적은 약 9,826,675 제곱 킬로미터 입니다.'

`combined` 객체의 `invoke` 메서드는 주어진 `country`에 대한 처리를 수행합니다.

이 예제에서는 `대한민국`라는 주제를 `invoke` 메서드에 전달하여 실행합니다.


In [19]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 총 면적은 약 100,363km² 입니다.'}

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다.
배치와 병렬 처리를 사용해 보도록 합시다.


`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.


In [20]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울입니다.', '미국의 수도는 워싱턴 D.C.입니다.']

`chain2.batch` 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 일괄 처리(batch)를 수행합니다.

이 예시에서는 `대한민국`와 `미국`라는 두 가지 국가에 대한 처리를 요청합니다.


In [21]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 총 면적은 약 100,363.4km² 입니다.', '미국의 면적은 약 9,833,520 km² 입니다.']

`combined.batch` 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 `대한민국`와 `미국` 두 나라에 대한 데이터를 배치 처리합니다.


In [22]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 총 면적은 약 100,363 km² 입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 면적은 약 9,834만 제곱 킬로미터 입니다.'}]